In [19]:
import pandas as pd
# from azure.ai.ml import MLClient
# from azure.identity import DefaultAzureCredential
from sklearn.model_selection import train_test_split

# ml_client = MLClient.from_config(credential=DefaultAzureCredential())
# data_asset = ml_client.data.get("allo_cine", version="1.1")

df = pd.read_parquet("allocine_spider_clean.parquet")
df.head()

,actors,critics_score,date,director,editor,genre,langage,length,nationality,viewers_score,french_first_week_boxoffice,french_visa,title,vo_title,url
0,"[George Clooney, Julia Roberts, Jack O'Connell]",3.5,2016-05-12,Jodie Foster,Sony Pictures Releasing France,[Thriller],[Anglais],99.0,[U.S.A.],3.6,305385.0,144018,Money Monster,None,/video/player_gen_cmedia=19561526&cfilm=214139...
1,"[Kad Merad, Géraldine Pailhas, Lola Creton]",2.7,2015-01-21,Christophe Lamotte,Rezo Films,"[Drame, Thriller]",[Français],100.0,[France],2.3,29265.0,117439,Disparue en hiver,None,/article/fichearticle_gen_carticle=18639324.html
2,"[Nicolas Cage, Kev Adams, Ryan Reynolds]",3.8,2013-04-10,Chris Sanders,Twentieth Century Fox France,"[Aventure, Animation, Comédie, Famille]",[Anglais],98.0,[U.S.A.],3.9,501465.0,135882,Les Croods,The Croods,/video/player_gen_cmedia=19410286&cfilm=146916...
3,"[Dylan O'Brien, Michael Keaton, Taylor Kitsch]",2.3,2017-09-20,Michael Cuesta,Metropolitan FilmExport,"[Action, Thriller]",[Anglais],112.0,[U.S.A.],3.2,104402.0,147308,American Assassin,None,/video/player_gen_cmedia=19569927&cfilm=194970...
4,"[Virginie Efira, Anaïs Demoustier, Laurent Sto...",3.5,2015-04-22,Emmanuel Mouret,Pyramide Distribution,"[Comédie, Romance]",[Français],100.0,[France],2.7,64046.0,138835,Caprice,None,/diaporamas/cinema/diaporama-18643599/


In [20]:
df = df[df['french_first_week_boxoffice'].notna()]

In [21]:
df.head()

,actors,critics_score,date,director,editor,genre,langage,length,nationality,viewers_score,french_first_week_boxoffice,french_visa,title,vo_title,url
0,"[George Clooney, Julia Roberts, Jack O'Connell]",3.5,2016-05-12,Jodie Foster,Sony Pictures Releasing France,[Thriller],[Anglais],99.0,[U.S.A.],3.6,305385.0,144018,Money Monster,None,/video/player_gen_cmedia=19561526&cfilm=214139...
1,"[Kad Merad, Géraldine Pailhas, Lola Creton]",2.7,2015-01-21,Christophe Lamotte,Rezo Films,"[Drame, Thriller]",[Français],100.0,[France],2.3,29265.0,117439,Disparue en hiver,None,/article/fichearticle_gen_carticle=18639324.html
2,"[Nicolas Cage, Kev Adams, Ryan Reynolds]",3.8,2013-04-10,Chris Sanders,Twentieth Century Fox France,"[Aventure, Animation, Comédie, Famille]",[Anglais],98.0,[U.S.A.],3.9,501465.0,135882,Les Croods,The Croods,/video/player_gen_cmedia=19410286&cfilm=146916...
3,"[Dylan O'Brien, Michael Keaton, Taylor Kitsch]",2.3,2017-09-20,Michael Cuesta,Metropolitan FilmExport,"[Action, Thriller]",[Anglais],112.0,[U.S.A.],3.2,104402.0,147308,American Assassin,None,/video/player_gen_cmedia=19569927&cfilm=194970...
4,"[Virginie Efira, Anaïs Demoustier, Laurent Sto...",3.5,2015-04-22,Emmanuel Mouret,Pyramide Distribution,"[Comédie, Romance]",[Français],100.0,[France],2.7,64046.0,138835,Caprice,None,/diaporamas/cinema/diaporama-18643599/


In [22]:
features_of_interest = [
    'actors',
    'date',
    'director',
    'editor',
    'genre',
    'langage',
    'length',
    'nationality',
]

info_film = ['french_visa', 'title', 'vo_title', 'url']

numerical_features = ['length']
date_feature = ['date']
categorical_features = ['director', 'editor']
list_categorical_features = ['actors', 'genre', 'langage', 'nationality']


list_categorical_features = ['actors', 'genre', 'langage', 'nationality']
for col in list_categorical_features:
    df[col] = df[col].mask(df[col].isna(), ['no value'])

target = 'french_first_week_boxoffice'

X, y = (
    df[features_of_interest],
    df[target]
)

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.1, random_state=42)

In [23]:
df = df.sort_values('date')
train_size = int(len(df) * 0.9)
train_df = df.iloc[:train_size]
test_df = df.iloc[train_size:]
X_train = train_df[features_of_interest]
y_train = train_df[target]

X_test = test_df[features_of_interest]
y_test = test_df[target]

# X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.1, random_state=42)

In [24]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MultiLabelBinarizer, FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
import numpy as np

class MultiLabelBinarizerTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.mlbs = {}  # Stocke un MultiLabelBinarizer pour chaque colonne
    
    def fit(self, X, y=None):
        for col in X.columns:
            self.mlbs[col] = MultiLabelBinarizer()
            self.mlbs[col].fit(X[col])
        return self
    
    def transform(self, X):
        transformed_list = []
        for col in X.columns:
            transformed = self.mlbs[col].transform(X[col])
            new_columns = [f"{col}_{label}" for label in self.mlbs[col].classes_]
            transformed_list.append(pd.DataFrame(transformed, columns=new_columns, index=X.index))
        
        return pd.concat(transformed_list, axis=1)
    
    def get_feature_names_out(self, input_features=None):
        # Collecter tous les noms de colonnes de sortie
        feature_names = []
        for col in self.mlbs.keys():
            feature_names.extend([f"{col}_{label}" for label in self.mlbs[col].classes_])
        return np.array(feature_names)

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')), # Bourin... à changer mais j'ai même pas vérifier s'il manquait des données...
    ('scaler', StandardScaler())
])

class CustomDateTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.feature_names_out = ['year', 'month', 'day', 'dayofweek']
        
    def fit(self, X, y=None):
        return self
        
    def transform(self, X):
        result = pd.DataFrame({
            'year': X['date'].dt.year,
            'month': X['date'].dt.month,
            'day': X['date'].dt.day,
            'dayofweek': X['date'].dt.dayofweek
        })
        return result
        
    def get_feature_names_out(self, input_features=None):
        return np.array(self.feature_names_out)

date_transformer = Pipeline(steps=[
    ('date_features', CustomDateTransformer())
])

# Toujours bourin :p
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

list_categorical_transformer = Pipeline(steps=[
    ('multi_label', MultiLabelBinarizerTransformer())
])

# Création du préprocesseur (sans les colonnes déjà transformées)
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('date', date_transformer, date_feature),
        ('cat', categorical_transformer, categorical_features),
        ('list', list_categorical_transformer, list_categorical_features)
    ],
    remainder='passthrough'
)

In [25]:
import xgboost as xgb
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
xgboost_model = Pipeline([
    ('preprocessor', preprocessor),  
    ('xgboost', xgb.XGBRegressor())
])

xgboost_model.fit(X_train,y_train)
y_pred = xgboost_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae}")
print(f"MAPE: {mape} %")
print(f"RMSE: {rmse}")
print(f"R²: {r2}")

MAE: 150753.2615712963
MAPE: 69.25406992449003 %
RMSE: 984751.4650020037
R²: 0.03801156547048368


/home/utilisateur/Documents/Simplon/new_is_always_better/.venv/lib/python3.12/site-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/home/utilisateur/Documents/Simplon/new_is_always_better/.venv/lib/python3.12/site-packages/sklearn/preprocessing/_label.py:909: UserWarning: unknown class(es) [' .jU.', ' Cali', ' Kaimana', ' Kanichiro', ' Kavin', ' Koyuki', ' Lal', ' Li Xuejian', ' Marie-Mai', ' Piqa', ' Samuthirakani', ' Siddique', ' Sneha', ' Soso Maness', ' Sunil', ' Tabu', ' Tai-Bo', 'Aaditi Pohankar', 'Aaron Pierre', 'Abdallah Charki', 'Abeer Mohammed', 'Abhay Verma', 'Abou Sangaré', 'Adain Bradley', 'Addison Rae', 'Aditya Nanda', 'Adrien Antoine', 'Agnès Sourdillon', 'Ahilleas Hariskos', 'Ai Kakuma', 'Aisa Winter', 'Aitziber Garmendia', 'Aixa Kay', 'Akira Kamiya', '

In [26]:
list_categorical_features = ['genre', 'langage', 'nationality', 'actors']
for col in list_categorical_features:
    df[col] = df[col].mask(df[col].isna(), ['no value'])
    
df['french_prod']= df['nationality'].apply(lambda x: 1 if "France" in x else 0)
df['usa_prod']= df['nationality'].apply(lambda x: 1 if "U.S.A." in x else 0)


list_categorical_features = ['genre', 'langage']
features_of_interest = [
    'french_prod',
    'date',
    'director',
    'editor',
    'genre',
    'langage',
    'length',
    'usa_prod',
]

info_film = ['french_visa', 'title', 'vo_title', 'url']

numerical_features = ['length']
date_feature = ['date']
categorical_features = ['director', 'editor']
list_categorical_features = ['genre', 'langage']


target = 'french_first_week_boxoffice'

X, y = (
    df[features_of_interest],
    df[target]
)
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.1, random_state=42)

In [27]:
import xgboost as xgb
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('date', date_transformer, date_feature),
        ('cat', categorical_transformer, categorical_features),
        ('list', list_categorical_transformer, list_categorical_features)
    ],
    remainder='passthrough'
)

xgboost_model = Pipeline([
    ('preprocessor', preprocessor),  
    ('xgboost', xgb.XGBRegressor())
])

xgboost_model.fit(X_train,y_train)
y_pred = xgboost_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae}")
print(f"RMSE: {rmse}")
print(f"R²: {r2}")

MAE: 119334.23865604529
RMSE: 702927.7454086209
R²: 0.10353007590984886


/home/utilisateur/Documents/Simplon/new_is_always_better/.venv/lib/python3.12/site-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/home/utilisateur/Documents/Simplon/new_is_always_better/.venv/lib/python3.12/site-packages/sklearn/preprocessing/_label.py:909: UserWarning: unknown class(es) ['Concert'] will be ignored
  warnings.warn(
/home/utilisateur/Documents/Simplon/new_is_always_better/.venv/lib/python3.12/site-packages/sklearn/preprocessing/_label.py:909: UserWarning: unknown class(es) ['Galicien'] will be ignored
  warnings.warn(
/home/utilisateur/Documents/Simplon/new_is_always_better/.venv/lib/python3.12/site-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods